In [4]:
# import libraries
import re
import nltk
import math
import contractions
from collections import defaultdict
from difflib import get_close_matches, SequenceMatcher

In [5]:
 nltk.download('punkt')
 nltk.download('punkt_tab')
 !pip install contractions

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Data Downloading

In [6]:
file_path = "/content/tedtalk.txt"
with open(file_path, 'r', encoding = 'utf-8') as f:
  data = f.read()

### Data Preprocessing

In [7]:
# preprocessing data
def preprocess_text(data):
  # convert to lowercase
  text = data.lower()
  # separate text into sentences
  sentences = nltk.sent_tokenize(text)
  processed_stc = []

  for sentence in sentences:
    # expand contractions
    sentence = contractions.fix(sentence)
    # remove punctuation except ., !, ?
    sentence = re.sub(r"[^a-z0-9.!?]", " ", sentence)
    # remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    # tokenize using nltk
    tokens = nltk.word_tokenize(sentence)

    # add <s> at the beginning and </s> at the end for sentence boundary
    processed_stc.append(["<s>"] + tokens + ["</s>"])

  return processed_stc

# processed_data
processed_data = preprocess_text(data)

### a. Improve the model by using interpolation smoothing with the "Stupid Backoff" method

In [8]:
class StupidBackoffModel:
  # n (int): Highest n-gram order
  # alpha (float): Backoff discount factor (typically 0.4 as Brants et al., 2007)
    def __init__(self, n=3, alpha=0.4):
        self.n = n
        self.alpha = alpha  # Discount factor for backoff
        # Stores n-gram counts: index 0=unigrams, 1=bigrams, 2=trigrams
        self.ngrams = [defaultdict(int) for _ in range(n)]
        self.vocab = set()  # Unique words in the corpus
        self.total_words = 0  # Total tokens

    # train the model
    def train(self, processed_data):
        for sentence in processed_data:
            self.total_words += len(sentence)
            for i in range(len(sentence)):
                word = sentence[i]
                # Count unigrams (order 0)
                self.ngrams[0][(word,)] += 1
                self.vocab.add(word)
                # Count bigrams (order 1) if there is a previous word
                if i >= 1:
                    bigram = tuple(sentence[i-1:i+1])
                    self.ngrams[1][bigram] += 1
                # Count trigrams (order 2) if there are two previous words
                if i >= 2:
                    trigram = tuple(sentence[i-2:i+1])
                    self.ngrams[2][trigram] += 1

    # Computes the score for a candidate word given a context using Stupid Backoff
    def score(self, context, candidate):
        current_context = list(context)
        accumulated_alpha = 1.0

        # check trigram to unigram
        max_order = min(len(current_context), self.n - 1)
        for order in range(max_order, 0, -1):
            ngram = tuple(current_context[-order:] + [candidate])
            count = self.ngrams[order].get(ngram, 0)

            if count > 0:
                prefix = tuple(current_context[-order:])
                prefix_count = self.ngrams[order-1].get(prefix, 0) if order > 0 else self.total_words
                if prefix_count == 0:
                    continue
                return accumulated_alpha * (count / prefix_count)

            # Backoff: reduce context and accumulate alpha
            current_context = current_context[1:]
            accumulated_alpha *= self.alpha

        # Fallback to unigram
        unigram_count = self.ngrams[0].get((candidate,), 0)
        return accumulated_alpha * (unigram_count / self.total_words)


### Train model

In [9]:
#train model
model = StupidBackoffModel(n=3, alpha=0.4)
model.train(processed_data)

### b. Compare with the results from In Class Exercise

Compare with the results from in-class exercise using sentence probability and perplexity. The results from in-class exercise:

1-gram probability: 1.3281553436964184e-17, 1-gram perplexity: 128.70454711879646

2-gram probability: 5.088333895898929e-16, 2-gram perplexity: 81.59772486118717

3-gram probability: 1.8867762518029128e-16, 3-gram perplexity: 92.37085208501146

In [10]:
# calculate probability
def calculate_sentence_probability(model, processed_sentence):
    tokens = processed_sentence
    probability = 1.0

    # Iterate from the starting position with sufficient context (n-1 words)
    for i in range(model.n - 1, len(tokens)):
        context = tokens[i - (model.n - 1):i]  # context
        word = tokens[i]
        prob = model.score(context, word)
        probability *= prob if prob > 0 else 1e-10  # avoid zero

    return probability

In [11]:
# calculate perplexity
def calculate_perplexity(model, processed_sentence):
    m = len(processed_sentence)
    prob = calculate_sentence_probability(model, processed_sentence)
    return math.pow(prob, -1/m)

In [12]:
# example sentence
ex = "I have to go to school"
ex = preprocess_text(ex)
ex = [word for sentence in ex for word in sentence]

print(ex)

prob_trigram = calculate_sentence_probability(model, ex)
perplexity_trigram = calculate_perplexity(model, ex)
print(f"[Trigram Stupid Backoff]")
print(f"Probability: {prob_trigram:.10f}")
print(f"Perplexity: {perplexity_trigram:.2f}\n")

['<s>', 'i', 'have', 'to', 'go', 'to', 'school', '</s>']
[Trigram Stupid Backoff]
Probability: 0.0000000156
Perplexity: 9.46



### Comparsion:

#### **1. Probability Comparison**  
- **Laplace Smoothing:**  
  - 1-gram: $$ 1.33 \times 10^{-17} $$  
  - 2-gram: $$ 5.09 \times 10^{-16} $$
  - 3-gram: $$ 1.89 \times 10^{-16} $$  
- **Stupid Backoff (Trigram-based):**  
  - Probability: $$ 1.56 \times 10^{-8} $$

The probability from Stupid Backoff is higher than all Laplace-based probabilities. This happens because Laplace smoothing assigns nonzero probability to unseen n-grams, which tends to over-smooth the distribution, leading to lower overall probability estimates. In contrast, Stupid Backoff prioritizes longer n-grams first and only backs off when necessary, leading to relatively higher probability estimates.

#### **2. Perplexity Comparison**  
- **Laplace Smoothing:**  
  - 1-gram: 128.70  
  - 2-gram: 81.60  
  - 3-gram: 92.37  
- **Stupid Backoff (Trigram-based):**  
  - Perplexity: 9.46  

The Stupid Backoff model results in significantly lower perplexity than all Laplace smoothing models. Since perplexity measures how well the model predicts the test sentence, a lower perplexity means a better predictive model. Stupid Backoff generally performs better because it utilizes higher-order n-grams more effectively rather than applying uniform smoothing across all n-grams.


### c. Generate the next words for a given word sequence

In [13]:
# Generate the top-k next words for a given context using the Stupid Backoff model
def generate_next_words(model, context, top_k=5): # specify top words = 5
    candidates = []
    for word in model.vocab:
        score = model.score(context, word)
        candidates.append((word, score))

    # Sort by score (descending) and return top-k
    candidates.sort(key=lambda x: -x[1])
    return candidates[:top_k]

In [14]:
# Example
context = ["the", "speaker", "is"]
next_words = generate_next_words(model, context, top_k=5)
print(f"Next words after '{' '.join(context)}':")
for word, score in next_words:
    print(f"- {word}: {score:.4f}")

Next words after 'the speaker is':
- to: 0.2222
- going: 0.2222
- trying: 0.1111
- saying: 0.1111
- an: 0.1111


### d. Combine with a function that calculates the distance between words to predict the correct word for a misspelled word position

In [15]:
# Correct a misspelled word in a sentence using similarity and language model scores
def correct_spelling(model, sentence, error_position, num_candidates=3, cutoff=0.6):

    tokens = preprocess_text(sentence)

    tokens = [word for sentence in tokens for word in sentence]
    tokens = [word for word in tokens if word not in {"<s>", "</s>"}]

    misspelled_word = tokens[error_position]

    # generate spelling candidates using difflib
    candidates = get_close_matches(misspelled_word, model.vocab, n=num_candidates, cutoff=cutoff)

    if not candidates:
        return sentence  # no correction possible

    # get context
    context_start = max(0, error_position - (model.n - 1))
    context = tokens[context_start:error_position]

    # using both similarity and LM score to find candidate
    best_candidate = None
    best_score = -1
    for candidate in candidates:
        # Language model score
        lm_score = model.score(context, candidate)
        # Similarity score (0-1)
        similarity = SequenceMatcher(None, misspelled_word, candidate).ratio()
        # Combined score
        combined_score = lm_score * similarity
        if combined_score > best_score:
            best_score = combined_score
            best_candidate = candidate

    # replace the misspelled word
    tokens[error_position] = best_candidate
    return ' '.join(tokens)

In [16]:
# example
ex = "I hae to go to schol"
# start with 0
corrected = correct_spelling(model, ex, error_position=1)  # Correct "hae" → "have"
corrected = correct_spelling(model, corrected, error_position=5)  # Correct "schol" → "school"
print(f"Original: {ex}")
print(f"Corrected: {corrected}")

Original: I hae to go to schol
Corrected: i have to go to school
